In [1]:
'''
Population data from FRED API

'''

import requests
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime
plt.style.use('seaborn-whitegrid')
import numpy as np
import os
from dateutil import parser
from collections import defaultdict

In [2]:

def do_series_search(search_data):
    '''
    Does a search of FRED data series
    INPUT: data to seach (list of strings)
    OUTPUT: json results from FRED
    '''
    fred_search_url = 'https://api.stlouisfed.org/fred/series/search?api_key=4a35874b8a4526dd50b390e46f396651&search_text={0}&file_type=json'  
    if type(search_data) != type(list):
        search_data = [search_data]
    search_string = '+'.join([x.lower() for x in search_data])
    print(search_string)
    fred_response = requests.get(fred_search_url.format(search_string))
    result = fred_response.json()
    return result
    
def get_series(series_id):
    '''
    Fetches FRED data series
    INPUT: data to seach (list of strings)
    OUTPUT: json results from FRED
    '''
    fred_series_fetch_url = 'https://api.stlouisfed.org/fred/series/observations?series_id={0}' + \
'&api_key=4a35874b8a4526dd50b390e46f396651&file_type=json'
    fred_response = requests.get(fred_series_fetch_url.format(series_id))
    result = fred_response.json()
    #print(result)
    return result 


def get_population_data(country):
    '''
    Get Poplatation data from FRED
    INPUT: Country (String)
    OUTPUT: dictionary of data 
    '''
    population_titles_of_interest = [
    'Employment to Population Ratio for',
    'Age Dependency Ratio: Older Dependents to Working-Age Population for',
    'Population Ages 15 to 64 for',
    'Population, Total for',
    'Population'
    ]

    search_string = country.lower()
    
    country_series  = do_series_search(search_string)
    series_ids = {}
    series_notes = {}
    #print(country_series)

    for elem in country_series['seriess']:
        #print(elem)
        for title in population_titles_of_interest:
            #if elem['title'].startswith(title):
            if 'population' in elem['title'].lower():
                series_ids[elem['title']] = elem['id']
                series_notes[elem['title']] = elem['notes']
    print(series_ids)
    ans = {}
    for title, series_id in series_ids.items():
        series = {}
        series_info = get_series(series_id)
        series['title'] = title
        series['units'] = series_info['units']
        series['notes'] = series_notes[title]
        series['dates'] = [x['date'] for x in series_info['observations']]
        series['values'] = [x['value'] for x in series_info['observations']]
        ans[title] = series
    
    return ans


def get_series_info_by_title(title, country):
    '''
    Get series data by specific title
    INPUT: title (str) 
            country (str)
    OUTPUT: series info
    
    '''
    print(title)
    print(country)
    save_country = country
    if country.lower() == 'south korea':
        country = 'republic of korea'
    country_series  = do_series_search(country.lower())
    series_ids = {}
    series_notes = {}
    for elem in country_series['seriess']:
        print(elem['title'] ,elem['frequency'], elem['id'])
        #print(elem)
        if title.lower() == elem['title'].lower():
            if 'frequency' in elem:
                if elem['frequency'].lower() == 'annual':
                    print('found one {0}'.format(title))
                    print(elem)
                    series_ids[elem['title']] = elem['id']
                    series_notes[elem['title']] = elem['notes']
    print('series ids', series_ids)
    ans = {}
    # For some reason cannot generically find us data
    # series_id['population, total for united states'] = 'POPTOTUSA647NWDB'
    # series_notes = {'title':'population, total for united states'}
    for title, series_id in series_ids.items():
        print('search1', title, series_id)
        series = {}
        series_info = get_series(series_id)
        series['title'] = title
        series['units'] = series_info['units']
        series['notes'] = series_notes[title]
        series['dates'] = [x['date'] for x in series_info['observations']]
        series['values'] = [x['value'] for x in series_info['observations']]
        ans[save_country] = series
    return ans
    

        
                
    
    
    

In [3]:
'''
These do not work as the titles vary by country

'''

def get_total_population_data_for_counry_set(country_set):
    '''
    Get total population
    
    '''
    country_set_data = {}
    for country in country_set:
        save_country = country
        country = country.lower()
        title_string = 'population for {0}'.format(country)
        if country.lower() in {'usa', 'united states'}:
            title_string = 'population' 
        print(title_string)
        info = get_series_info_by_title(title_string, save_country)
        country_set_data[save_country] = info
    return country_set_data


'''


'''
def get_working_age_percent_for_country_set(country_set):
    country_set_data = {}
    for country in country_set:
        save_country = country
        country = country.lower()
        title_string = 'population ages 15 to 64 for {0}'.format(country)
        info = get_series_info_by_title(title_string, save_country)
        country_set_data[save_country] = info
    return country_set_data
    
    
    

In [4]:
this_set = get_total_population_data_for_counry_set(['United States'])





population
population
United States
united states
Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level Weekly, As of Wednesday WALCL
U.S. / Euro Foreign Exchange Rate Daily DEXUSEU
U.S. / Euro Foreign Exchange Rate Monthly EXUSEU
U.S. / Euro Foreign Exchange Rate Annual AEXUSEU
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANS
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCI
Assets: Securities Held Outright: U.S. Treasury Securities: All: Wednesday Level Weekly, As of Wednesday TREAST
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCINSA
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANSNSA
M2 Money Stock Weekly, Ending Monday M2
10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity Daily T10Y2Y
Unemployment Rate Monthly UNRATE
Consumer Price Index for All Urban Consumers: All Items in U.S. City Average Monthly CPIAUCS

Sweden / U.S. Foreign Exchange Rate Monthly EXSDUS
St. Louis Adjusted Monetary Base (DISCONTINUED) Biweekly, Ending Wednesday DSBASENS
Motor Vehicle Retail Sales: Heavy Weight Trucks Monthly HTRUCKSSA
New One Family Houses Sold: United States Monthly HSN1FNSA
Private Final Consumption Expenditure in United States Quarterly USAPFCEQDSNAQ
S&P/Case-Shiller CA-San Francisco Home Price Index Monthly SFXRNSA
Personal Income Quarterly PINCOME
Commercial and Industrial Loans, Large Domestically Chartered Commercial Banks Weekly, Ending Wednesday CIBOARDNSA
Sweden / U.S. Foreign Exchange Rate Annual AEXSDUS
Commercial and Industrial Loans, Large Domestically Chartered Commercial Banks Monthly CILLCBM027NBOG
Motor Vehicle Retail Sales: Heavy Weight Trucks Monthly HTRUCKSNSA
1-Year Treasury Bill: Secondary Market Rate Monthly TB1YR
Commercial and Industrial Loans, Large Domestically Chartered Commercial Banks Monthly CILLCBM027SBOG
Health Expenditures per Capita Annual HLTHSCPCHCSA
Unemployment L

Retail Sales: Used Car Dealers Monthly MRTSSM44112USN
Treasury and Agency Securities: Non-MBS, All Commercial Banks Weekly, Ending Wednesday TNMACBW027SBOG
Real Exports of Goods and Services Annual EXPGSCA
Private Nonresidential Fixed Investment Quarterly PNFI
Mortgage Debt Service Payments as a Percent of Disposable Personal Income Quarterly MDSP
Borrowings, All Commercial Banks Weekly H8B3094NCBD
Liabilities and Capital: Liabilities: Deposits: Other Deposits Held by Depository Institutions: Wednesday Level Weekly, As of Wednesday WLODLL
Interbank Loans, All Commercial Banks (DISCONTINUED) Monthly IBLACBM027NBOG
series ids {'Population': 'B230RC0A052NBEA'}
search1 Population B230RC0A052NBEA


In [5]:
this_set

{'United States': {'United States': {'title': 'Population',
   'units': 'lin',
   'notes': 'Population includes resident population plus armed forces overseas. The annual estimate is constructed as an average of monthly series https://fred.stlouisfed.org/series/POPTHM. \nBEA Account Code: B230RC',
   'dates': ['1929-01-01',
    '1930-01-01',
    '1931-01-01',
    '1932-01-01',
    '1933-01-01',
    '1934-01-01',
    '1935-01-01',
    '1936-01-01',
    '1937-01-01',
    '1938-01-01',
    '1939-01-01',
    '1940-01-01',
    '1941-01-01',
    '1942-01-01',
    '1943-01-01',
    '1944-01-01',
    '1945-01-01',
    '1946-01-01',
    '1947-01-01',
    '1948-01-01',
    '1949-01-01',
    '1950-01-01',
    '1951-01-01',
    '1952-01-01',
    '1953-01-01',
    '1954-01-01',
    '1955-01-01',
    '1956-01-01',
    '1957-01-01',
    '1958-01-01',
    '1959-01-01',
    '1960-01-01',
    '1961-01-01',
    '1962-01-01',
    '1963-01-01',
    '1964-01-01',
    '1965-01-01',
    '1966-01-01',
    '196

In [6]:
country_set = ['China', 'Japan', 'Republic of Korea', 'Germany', 'United States']

In [7]:
total_population_set = get_total_population_data_for_counry_set(country_set)

population for china
population for china
China
china
China / U.S. Foreign Exchange Rate Daily DEXCHUS
China / U.S. Foreign Exchange Rate Monthly EXCHUS
China / U.S. Foreign Exchange Rate Annual AEXCHUS
Gross Domestic Product for China Annual MKTGDPCNA646NWDB
Consumer Price Index: All Items for China Monthly CHNCPIALLMINMEI
M2 for China Monthly MYAGM2CNM189N
Consumer Price Index: All Items for China Annual CHNCPIALLAINMEI
Consumer Price Index: All Items for China Quarterly CHNCPIALLQINMEI
Current Price Gross Domestic Product in China Quarterly CHNGDPNQDSMEI
Current Price Gross Domestic Product in China Annual CHNGDPNADSMEI
Real GDP at Constant National Prices for China Annual RGDPNACNA666NRUG
M1 for China Monthly MYAGM1CNM189N
M1 for China Monthly MANMM101CNM189S
M1 for China Monthly MANMM101CNM189N
M1 for China Quarterly MANMM101CNQ189N
M1 for China Monthly MANMM101CNM657S
M1 for China Annual MANMM101CNA189S
M1 for China Annual MANMM101CNA657S
M1 for China Quarterly MANMM101CNQ189S
M1

Total Net Errors and Omissions of Balance of Payments for China (DISCONTINUED) Annual BPEOTT01CNA637N
Balance of payments BPM6: Financial account: Other investment: Other investment Net for China Quarterly CHNB6FAOI01NCCUQ
Financial Account: Net Direct Investment for China (DISCONTINUED) Annual BPFADI01CNA637N
Current Account Debit: Trade: Total Imports of Goods for China (DISCONTINUED) Annual BPDBTD01CNA637N
Current Account Debit: Trade: Total Imports of Goods for China (DISCONTINUED) Annual BPDBTD01CNA637S
Current Account Debit: Trade: Total Imports of Goods for China (DISCONTINUED) Quarterly BPDBTD01CNQ637N
Current Account Balance: Total Income for China (DISCONTINUED) Quarterly BPBLIN01CNQ637N
Current Account Balance: Total Income for China (DISCONTINUED) Annual BPBLIN01CNA637S
Current Account Balance: Total Income for China (DISCONTINUED) Annual BPBLIN01CNA637N
Balance of payments BPM6: Financial account: Other investment: Other investment Net acquisition of financial assets for C

Value of Exports to Province of China Taiwan from California Annual CATWNA052SCEN
Number of Identified Exporters to China from Louisiana Annual LACHNA475SCEN
Use of Financial Services Deposit Accounts: Non-life Insurance Accounts at Insurance Corporations for China, P.R.: Macao Annual MACFCAOFILNNUM
Geographical Outreach: Key Indicators, Credit Union and Financial Cooperative Branches Per 1000 Km2 for China, P.R.: Mainland Annual CHNFCBODUKNUM
Amount Outstanding of International Money Market instruments for All Issuers, Residence of Issuer in China (DISCONTINUED) Quarterly IMMIRIAOCN
Geographical Outreach: Number of Institutions, Other Financial Corporations, Other Financial Intermediaries for China, P.R.: Hong Kong Annual HKGFCIOFMNUM
Number of Identified Exporters to China from North Carolina Annual NCCHNA475SCEN
Number of Identified Exporters to China from District of Columbia Annual DCCHNA475SCEN
Value of Exports to Province of China Taiwan from District of Columbia Annual DCTWNA05

Share of Residual Trade and GDP Statistical Discrepancy at Current Purchasing Power Parities for Japan Annual CSHRCPJPA156NRUG
Balance of payments BPM6: Financial account: Financial account Net: Total for Japan Quarterly JPNB6FATT01CXCUQ
Broad Effective Exchange Rate for Japan Monthly NBJPBIS
Compensation of Employees in Japan Quarterly JPNCOMPQDSNAQ
Net Issues of International Debt Securities for Issuers in General Government Sector, All Maturities, Residence of Issuer in Japan Quarterly IDSGAMRINIJP
Leading Indicators OECD: Component series: Interest rate spread: Normalised for Japan Monthly JPNLOCOSINOSTSAM
Leading Indicators OECD: Leading indicators: CLI: Normalised for Japan Monthly JPNLOLITONOSTSAM
Consumption Share of Purchasing Power Parity Converted GDP Per Capita at constant prices for Japan Annual KCPPPGJPA156NUPN
Consumer Opinion Surveys: Confidence Indicators: Composite Indicators: European Commission and National Indicators for Japan Monthly CSCICP02JPM460S
Population age

Broad Money, Index for Japan (DISCONTINUED) Monthly MABMBM01JPM657S
Broad Money, Index for Japan (DISCONTINUED) Monthly MABMBM01JPM661S
Broad Money, Index for Japan (DISCONTINUED) Annual MABMBM01JPA657S
Broad Money, Index for Japan (DISCONTINUED) Monthly MABMBM01JPM659S
Broad Money, Index for Japan (DISCONTINUED) Quarterly MABMBM01JPQ661S
Broad Money, Index for Japan (DISCONTINUED) Annual MABMBM01JPA659S
Broad Money, Index for Japan (DISCONTINUED) Annual MABMBM01JPA661S
Broad Money, Index for Japan (DISCONTINUED) Quarterly MABMBM01JPQ657S
Broad Money, Index for Japan (DISCONTINUED) Quarterly MABMBM01JPQ659S
Total Net Errors and Omissions of Balance of Payments for Japan (DISCONTINUED) Annual BPEOTT01JPA636N
Total Net Errors and Omissions of Balance of Payments for Japan (DISCONTINUED) Quarterly BPEOTT01JPQ636N
Employment by Economic Activity: Industry Excluding Construction: All Persons for Japan Monthly LFEAINTTJPM647N
Employment by Economic Activity: Industry Excluding Construction: 

...

Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Actual rentals for housing: Total for the Republic of Korea Monthly KORCP040100IXOBM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Actual rentals for housing: Total for the Republic of Korea Monthly KORCP040100GPM
Consumer Price Index: Housing, water, electricity, gas and other fuels (COICOP 04): Actual rentals for housing: Total for the Republic of Korea Monthly KORCP040100GYM
Population for Republic of Korea Annual POPTTLKRA148NRUG
found one population for republic of korea
{'id': 'POPTTLKRA148NRUG', 'realtime_start': '2021-01-14', 'realtime_end': '2021-01-14', 'title': 'Population for Republic of Korea', 'observation_start': '1953-01-01', 'observation_end': '2017-01-01', 'frequency': 'Annual', 'frequency_short': 'A', 'units': 'Millions of Persons', 'units_short': 'Mil. of Persons', 'seasonal_adjustment': 'Not Seasonally Adjusted', 'seasonal_adjustment_short': 'NS

Reserve Assets for Germany (DISCONTINUED) Monthly XFORNC01DEM189N
Reserve Assets for Germany (DISCONTINUED) Monthly XFORSD01DEM194N
Reserve Assets for Germany (DISCONTINUED) Annual XFORSD01DEA194N
Reserve Assets for Germany (DISCONTINUED) Quarterly XFORNC01DEQ189N
Reserve Assets for Germany (DISCONTINUED) Quarterly XFORSD01DEQ194N
Real Effective Exchange Rates Based on Manufacturing Consumer Price Index for Germany Quarterly CCRETT01DEQ661N
Real Effective Exchange Rates Based on Manufacturing Consumer Price Index for Germany Annual CCRETT01DEA661N
Consumer Price Index: Food for Germany Quarterly DEUCPIFODQINMEI
Consumer Price Index: Food for Germany Annual DEUCPIFODAINMEI
Financial Account: Liabilities: Direct Foreign Investment in Reporting Country for Germany (DISCONTINUED) Quarterly BPFADI03DEQ636N
Financial Account: Liabilities: Direct Foreign Investment in Reporting Country for Germany (DISCONTINUED) Annual BPFADI03DEA636N
Financial Account: Liabilities: Direct Foreign Investment 

Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for Germany Quarterly DEUCP020000GPQ
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for Germany Quarterly DEUCP020000GYQ
Consumer Price Index: Alcoholic beverages, tobacco and narcotics (COICOP 02): Total: Total for Germany Quarterly DEUCP020000IXNBQ
Proceeds from Increases in Capital Stock for Germany Monthly M10025DEM421NNBR
Producer Prices Index: Economic activities: Manufacturing: Total for Germany Monthly DEUPIEAMP01GPM
Producer Prices Index: Economic activities: Manufacturing: Total for Germany Monthly DEUPIEAMP01GYM
Producer Prices Index: Economic activities: Manufacturing: Total for Germany Quarterly DEUPIEAMP01GYQ
Outstanding Domestic Public Debt Securities to GDP for Germany Annual DDDM04DEA156NWDB
Capital stock at Current Purchasing Power Parities for Germany Annual CKSPPPDEA666NRUG
Consumer Price Index: Communication (COICOP 08): Total: Tota

Capacity Utilization: Total Index Quarterly CAPUTLB50001SQ
Real Gross Private Domestic Investment Quarterly ND000335Q
Real Gross Private Domestic Investment Annual A006RL1A225NBEA
Moving 12-Month Total Vehicle Miles Traveled Monthly M12MTVUSM227NFWA
Student Loans Owned and Securitized, Outstanding Quarterly, End of Period SLOAS
Memorandum Items: Custody Holdings: Marketable U.S. Treasury Securities: Wednesday Level Weekly, As of Wednesday WMTSECL1
Unemployment Rate - 20 Yrs. & Over Monthly LNU04000024
Effective Federal Funds Volume Daily EFFRVOL
St. Louis Adjusted Monetary Base (DISCONTINUED) Monthly DISAMBNS
M1 Money Multiplier (DISCONTINUED) Biweekly, Ending Wednesday MULT
S&P/Case-Shiller CA-Los Angeles Home Price Index Monthly LXXRSA
NASDAQ Composite Index Daily, Close NASDAQCOM
Total Savings Deposits at all Depository Institutions Weekly, Ending Monday WSAVNS
Real Gross Private Domestic Investment Quarterly OB000335Q
Real Gross Private Domestic Investment Quarterly NB000335Q
St. L

Real Imports of Goods and Services Quarterly A021RL1Q158SBEA
3-Month AA Financial Commercial Paper Rate Weekly, Ending Friday WCPF3M
Personal Consumption Expenditures: Services Quarterly NA000350Q
Personal income per capita Quarterly A792RC0Q052SBEA
Unemployment Rate - White Monthly LNU04000003
Consumer Price Index for All Urban Consumers: Housing in U.S. City Average Monthly CPIHOSSL
Total Business Inventories Monthly TOTBUSMPCIMNSA
Personal Consumption Expenditures: Nondurable Goods Quarterly NC000348Q
Personal Consumption Expenditures: Services Quarterly NC000350Q
Real Imports of Goods and Services Annual A021RL1A158NBEA
Real Imports of Goods and Services Quarterly NB000342Q
Real Imports of Goods and Services Quarterly OB000342Q
Unemployment Rate: Aged 15-64: All Persons for the United States Annual LRUN64TTUSA156S
Unemployment Rate: Aged 15-64: All Persons for the United States Monthly LRUN64TTUSM156N
Total Construction Spending: Nonresidential Monthly TLNRESCON
S&P/Case-Shiller TX

In [8]:

total_population_set

{'China': {'China': {'title': 'Population for China',
   'units': 'lin',
   'notes': 'Source ID: pop\n\nWhen using these data in your research, please make the following reference: Feenstra, Robert C., Robert Inklaar and Marcel P. Timmer (2015), "The Next Generation of the Penn World Table" American Economic Review, 105(10), 3150-3182, available for download at www.ggdc.net/pwt\n\nFor more information, see http://www.rug.nl/research/ggdc/data/pwt/.',
   'dates': ['1952-01-01',
    '1953-01-01',
    '1954-01-01',
    '1955-01-01',
    '1956-01-01',
    '1957-01-01',
    '1958-01-01',
    '1959-01-01',
    '1960-01-01',
    '1961-01-01',
    '1962-01-01',
    '1963-01-01',
    '1964-01-01',
    '1965-01-01',
    '1966-01-01',
    '1967-01-01',
    '1968-01-01',
    '1969-01-01',
    '1970-01-01',
    '1971-01-01',
    '1972-01-01',
    '1973-01-01',
    '1974-01-01',
    '1975-01-01',
    '1976-01-01',
    '1977-01-01',
    '1978-01-01',
    '1979-01-01',
    '1980-01-01',
    '1981-01-0

In [9]:
working_population_set = get_working_age_percent_for_country_set(country_set)

population ages 15 to 64 for china
China
china
China / U.S. Foreign Exchange Rate Daily DEXCHUS
China / U.S. Foreign Exchange Rate Monthly EXCHUS
China / U.S. Foreign Exchange Rate Annual AEXCHUS
Gross Domestic Product for China Annual MKTGDPCNA646NWDB
Consumer Price Index: All Items for China Monthly CHNCPIALLMINMEI
M2 for China Monthly MYAGM2CNM189N
Consumer Price Index: All Items for China Annual CHNCPIALLAINMEI
Consumer Price Index: All Items for China Quarterly CHNCPIALLQINMEI
Current Price Gross Domestic Product in China Quarterly CHNGDPNQDSMEI
Current Price Gross Domestic Product in China Annual CHNGDPNADSMEI
Real GDP at Constant National Prices for China Annual RGDPNACNA666NRUG
M1 for China Monthly MYAGM1CNM189N
M1 for China Monthly MANMM101CNM189S
M1 for China Monthly MANMM101CNM189N
M1 for China Quarterly MANMM101CNQ189N
M1 for China Monthly MANMM101CNM657S
M1 for China Annual MANMM101CNA189S
M1 for China Annual MANMM101CNA657S
M1 for China Quarterly MANMM101CNQ189S
M1 for Ch

Total Credit to Private Non-Financial Sector, Unadjusted for Breaks, for China Quarterly, End of Quarter CRDQCNAPUBIS
External Trade: Volume of Exports of Goods and Services for China, P.R.: Hong Kong Annual HKGTXRPCPPPT
Pension Fund Assets to GDP for China Annual DDDI13CNA156NWDB
Leading Indicators OECD: Leading indicators: CLI: Trend restored for China Monthly CHNLOLITOTRGYSAM
Reserve Assets for China (DISCONTINUED) Annual XFORSD01CNA194N
Reserve Assets for China (DISCONTINUED) Quarterly XFORSD01CNQ194N
Broad Money, Index for China (DISCONTINUED) Monthly MABMBM01CNM661S
Broad Money, Index for China (DISCONTINUED) Monthly MABMBM01CNM659S
Broad Money, Index for China (DISCONTINUED) Annual MABMBM01CNA659S
Broad Money, Index for China (DISCONTINUED) Quarterly MABMBM01CNQ659S
Broad Money, Index for China (DISCONTINUED) Annual MABMBM01CNA661S
Broad Money, Index for China (DISCONTINUED) Annual MABMBM01CNA657S
Broad Money, Index for China (DISCONTINUED) Quarterly MABMBM01CNQ661S
Broad Money,

People 15 Years and Over Borrowing from a Financial Institution in the Past Year for China Annual DDAI07CNA156NWDB
Geographical Outreach: Number of Institutions, Other Financial Corporations, Insurance Corporations for China, P.R.: Hong Kong Annual HKGFCIOFINUM
Net Issues of International Bonds and Notes for All Issuers, Residence of Issuer in China (DISCONTINUED) Quarterly IBANRINICN
Amount Outstanding of International Money Market instruments for All Issuers, Nationality of Issuer in China (DISCONTINUED) Quarterly IMMINIAIAOCN
Number of Identified Exporters to China from Ohio Annual OHCHNA475SCEN
Number of Identified Exporters to China from Hawaii Annual HICHNA475SCEN
Number of Identified Exporters to China from Arizona Annual AZCHNA475SCEN
Number of Identified Exporters to China from Massachusetts Annual MACHNA475SCEN
Net Issues of International Debt Securities for Issuers in Other Financial Corporations, All Maturities, Residence of Issuer in China (DISCONTINUED) Quarterly IDSOFAMR

Consumer Price Index: Energy for Japan Quarterly JPNCPIENGQINMEI
Sales: Retail trade: Total retail trade: Volume for Japan Monthly JPNSLRTTO01GPSAM
Financial Account: Liabilities: Direct Foreign Investment in Reporting Country for Japan (DISCONTINUED) Quarterly BPFADI03JPQ636N
Financial Account: Liabilities: Direct Foreign Investment in Reporting Country for Japan (DISCONTINUED) Annual BPFADI03JPA636N
Financial Account: Liabilities: Direct Foreign Investment in Reporting Country for Japan (DISCONTINUED) Annual BPFADI03JPA637N
International Trade: Exports: Value (goods): Total for Japan Quarterly JPNXTEXVA01CXMLQ
Manufacturing Inventories to Shipments Ratio for Japan Quarterly STMNIS01JPQ661S
Manufacturing Inventories to Shipments Ratio for Japan Quarterly STMNIS01JPQ661N
Manufacturing Inventories to Shipments Ratio for Japan Annual STMNIS01JPA661N
Current Account Balance: Total Income for Japan (DISCONTINUED) Annual BPBLIN01JPA636N
Real Exports of Goods and Services for Japan Quarterly

Employment by Economic Activity: Agriculture: All Persons for Japan Monthly LFEAAGTTJPM647N
Employment by Economic Activity: Agriculture: All Persons for Japan Quarterly LFEAAGTTJPQ647S
Employment by Economic Activity: Agriculture: All Persons for Japan Annual LFEAAGTTJPA647N
Employment by Economic Activity: Agriculture: All Persons for Japan Quarterly LFEAAGTTJPQ647N
Employment by Economic Activity: Agriculture: All Persons for Japan Annual LFEAAGTTJPA647S
Working Age Population: Aged 15-64: Females for Japan Monthly LFWA64FEJPM647S
Working Age Population: Aged 15-64: Females for Japan Monthly LFWA64FEJPM647N
Working Age Population: Aged 15-64: Females for Japan Quarterly LFWA64FEJPQ647N
Working Age Population: Aged 15-64: Females for Japan Quarterly LFWA64FEJPQ647S
Working Age Population: Aged 15-64: Females for Japan Annual LFWA64FEJPA647N
Use of Financial Services, Assets: Outstanding Loans at Commercial Banks for Japan Annual JPNFCSODCXDC
Benchmarked Value Added - Manufacturing fo

...

Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Monthly IRLTLT01DEM156N
Real Gross Domestic Product for Germany Quarterly CLVMNACSCAB1GQDE
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Quarterly IRLTLT01DEQ156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Annual IRLTLT01DEA156N
Real Gross Domestic Product for Germany Quarterly CLVMNACNSAB1GQDE
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156S
Registered Unemployment Rate for Germany Quarterly LMUNRRTTDEQ156S
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156N
Consumer Price Index of All Items in Germany Monthly DEUCPIALLMINMEI
Registered Unemployment Rate for Germany Annual LMUNRRTTDEA156N
Registered Unemployment Rate for Germany Quarterly LMUNRRTTDEQ156N
Registered Unemployment Rate for Germany Annual LMUNRRTTDEA156S
Consumer Price Index of All Items in Germany Quarterly DEUCPIALLQINMEI
Gross Domestic 

Producer Prices Index: Total Consumer Goods for Germany Annual PITGCG01DEA661N
Permits Issued for Dwelling in Germany Annual DEUPERMITAISMEI
Permits Issued for Dwelling in Germany Quarterly DEUPERMITQISMEI
Current Account Balance: Total Trade of Goods for Germany (DISCONTINUED) Annual BPBLTD01DEA636N
Current Account Balance: Total Trade of Goods for Germany (DISCONTINUED) Annual BPBLTD01DEA637S
Current Account Balance: Total Trade of Goods for Germany (DISCONTINUED) Quarterly BPBLTD01DEQ636S
Current Account Balance: Total Trade of Goods for Germany (DISCONTINUED) Annual BPBLTD01DEA636S
Current Account Balance: Total Trade of Goods for Germany (DISCONTINUED) Quarterly BPBLTD01DEQ636N
Reserve Assets for Germany (DISCONTINUED) Monthly XFORNC01DEM189N
Reserve Assets for Germany (DISCONTINUED) Monthly XFORSD01DEM194N
Reserve Assets for Germany (DISCONTINUED) Annual XFORSD01DEA194N
Reserve Assets for Germany (DISCONTINUED) Quarterly XFORNC01DEQ189N
Reserve Assets for Germany (DISCONTINUED) Q

Total Credit to Non-Financial Corporations, Adjusted for Breaks, for Germany Quarterly, End of Quarter CRDQDEANABIS
Leading Indicators OECD: Component Series: Business Tendency Survey: Demand or Orders Inflow: Original Series for Germany Quarterly LOCOBDORDEQ460S
Potato Crop for Germany Annual A0116BDEA391NNBR
Potato Crop for Germany Annual A0116CDEA391NNBR
Potato Crop for Germany Annual A0116ADEA391NNBR
Potato Crop for Germany Annual A01016DEA391NNBR
International Trade: Net trade: Value (goods): Total for Germany Monthly DEUXTNTVA01CXMLM
International Trade: Net trade: Value (goods): Total for Germany Quarterly DEUXTNTVA01STSAQ
International Trade: Net trade: Value (goods): Total for Germany Quarterly DEUXTNTVA01CXMLQ
Employment to Population Rate: All Ages: All Persons for Germany Quarterly LREPTTTTDEQ156N
Employment to Population Rate: All Ages: All Persons for Germany Quarterly LREPTTTTDEQ156S
Employment to Population Rate: All Ages: All Persons for Germany Annual LREPTTTTDEA156N


Job Openings: Total Nonfarm Monthly JTSJOL
Real Disposable Personal Income: Per Capita Annual A229RX0A048NBEA
Existing Home Sales Monthly EXHOSLUSM495N
1-Month Treasury Constant Maturity Rate Weekly, Ending Friday WGS1MO
Nonfinancial Corporate Business; Debt Securities; Liability, Level Annual NCBDBIA027N
St. Louis Adjusted Monetary Base (DISCONTINUED) Biweekly, Ending Wednesday BASENS
Capacity Utilization: Total Index Quarterly CAPUTLB50001SQ
Real Gross Private Domestic Investment Quarterly ND000335Q
Real Gross Private Domestic Investment Annual A006RL1A225NBEA
Moving 12-Month Total Vehicle Miles Traveled Monthly M12MTVUSM227NFWA
Student Loans Owned and Securitized, Outstanding Quarterly, End of Period SLOAS
Memorandum Items: Custody Holdings: Marketable U.S. Treasury Securities: Wednesday Level Weekly, As of Wednesday WMTSECL1
Unemployment Rate - 20 Yrs. & Over Monthly LNU04000024
Effective Federal Funds Volume Daily EFFRVOL
St. Louis Adjusted Monetary Base (DISCONTINUED) Monthly DIS

Coinbase Bitcoin Cash Daily, 7-Day CBBCHUSD
S&P/Case-Shiller CO-Denver Home Price Index Monthly DNXRNSA
Unemployment Rate in Ohio Annual LAUST390000000000003A
Personal Consumption Expenditures: Nondurable Goods Annual PCNDA
S&P/Case-Shiller NV-Las Vegas Home Price Index Monthly LVXRNSA
S&P/Case-Shiller FL-Miami Home Price Index Monthly MIXRNSA
Producer Price Index by Commodity: Metals and Metal Products: Titanium and Titanium-Base Alloy Mill Shapes Monthly WPU10250505
Unemployment Rate: Aged 15-64: All Persons for the United States Quarterly LRUN64TTUSQ156N
Total Business Inventories Monthly TOTBUSMPCIMSA
Real Imports of Goods and Services Quarterly ND000342Q
Personal income per capita Annual A792RC0A052NBEA
Trade Weighted U.S. Dollar Index: Emerging Markets Economies, Goods and Services Monthly TWEXEMEGSMTH
Personal Consumption Expenditures: Nondurable Goods Quarterly NA000348Q
6-Month Treasury Constant Maturity Rate Monthly GS6M
Total Business Inventories Monthly, End of Period TOTBU

In [10]:
working_population_set

{'China': {'China': {'title': 'Population Ages 15 to 64 for China',
   'units': 'lin',
   'notes': 'Source Indicator: SP.POP.1564.TO.ZS',
   'dates': ['1950-01-01',
    '1951-01-01',
    '1952-01-01',
    '1953-01-01',
    '1954-01-01',
    '1955-01-01',
    '1956-01-01',
    '1957-01-01',
    '1958-01-01',
    '1959-01-01',
    '1960-01-01',
    '1961-01-01',
    '1962-01-01',
    '1963-01-01',
    '1964-01-01',
    '1965-01-01',
    '1966-01-01',
    '1967-01-01',
    '1968-01-01',
    '1969-01-01',
    '1970-01-01',
    '1971-01-01',
    '1972-01-01',
    '1973-01-01',
    '1974-01-01',
    '1975-01-01',
    '1976-01-01',
    '1977-01-01',
    '1978-01-01',
    '1979-01-01',
    '1980-01-01',
    '1981-01-01',
    '1982-01-01',
    '1983-01-01',
    '1984-01-01',
    '1985-01-01',
    '1986-01-01',
    '1987-01-01',
    '1988-01-01',
    '1989-01-01',
    '1990-01-01',
    '1991-01-01',
    '1992-01-01',
    '1993-01-01',
    '1994-01-01',
    '1995-01-01',
    '1996-01-01',
    '19

In [11]:

working_age_population_set = {
'Germany':'Working Age Population: Aged 15-64: All Persons for Germany',
'South Korea':'Working Age Population: Aged 15-64: All Persons for the Republic of Korea',
'Japan':'Working Age Population: Aged 15-64: All Persons for Japan',
'USA':'Working Age Population: Aged 15-64: All Persons for the United States',
'China':'Population Ages 15 to 64 for China', # This is a ratio -- need to multiple by population
}

working_age_population_ratio_set = {
'Germany':'Employment to Population Ratio for Germany',
'South Korea':'Employment to Population Ratio for the Republic of Korea',
'Japan':'Employment to Population Ratio for Japan',
'USA':'Employment to Population Ratio for the United States', # USA
'China':'Population Ages 15 to 64 for China',
}


In [12]:
working_age_population = {}
for country in working_age_population_set:
    working_age_population[country] =  get_series_info_by_title( working_age_population_set[country], country )

Working Age Population: Aged 15-64: All Persons for Germany
Germany
germany
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Monthly IRLTLT01DEM156N
Real Gross Domestic Product for Germany Quarterly CLVMNACSCAB1GQDE
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Quarterly IRLTLT01DEQ156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Annual IRLTLT01DEA156N
Real Gross Domestic Product for Germany Quarterly CLVMNACNSAB1GQDE
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156S
Registered Unemployment Rate for Germany Quarterly LMUNRRTTDEQ156S
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156N
Consumer Price Index of All Items in Germany Monthly DEUCPIALLMINMEI
Registered Unemployment Rate for Germany Annual LMUNRRTTDEA156N
Registered Unemployment Rate for Germany Quarterly LMUNRRTTDEQ156N
Registered Unemployment Rate for Germany Annual LMUNRRTTDEA156S
Consumer P

Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Quarterly ULQBBU04DEQ661S
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Annual ULQBBU04DEA662N
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Annual DEUULCCONAPNMEI
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Annual ULQBBU04DEA657S
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Annual ULQBBU04DEA661N
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Annual ULQBBU04DEA661S
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Quarterly ULQBBU04DEQ661N
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Quarterly ULQBBU04DEQ662N
Unemployment Rate: Aged 25-54: Males for Germany Quarterly LRUN25MADEQ156S
Unemployment Rate: Aged 25-54: Males for Germany Quarterly LRUN25MADEQ156N
Unemployment Rate: Aged 25-54: Males for Germany Annual LRUN25MADEA156S
Harmonized Unempl

Private Discount Rate, Prime Banker's Acceptance, Open Market for Berlin, Germany Monthly M13018DE00BERM156NNBR
Loans and Discounts, All Issuing Banks for Germany Monthly M14091DEM421NNBR
Total Credit to Non-Financial Corporations, Adjusted for Breaks, for Germany Quarterly, End of Quarter QDENAM770A
Total Credit to Non-Financial Corporations, Adjusted for Breaks, for Germany Quarterly, End of Quarter QDENAMUSDA
Total Credit to Non-Financial Corporations, Adjusted for Breaks, for Germany Quarterly, End of Quarter CRDQDEANABIS
Leading Indicators OECD: Component Series: Business Tendency Survey: Demand or Orders Inflow: Original Series for Germany Quarterly LOCOBDORDEQ460S
Potato Crop for Germany Annual A0116BDEA391NNBR
Potato Crop for Germany Annual A0116CDEA391NNBR
Potato Crop for Germany Annual A0116ADEA391NNBR
Potato Crop for Germany Annual A01016DEA391NNBR
International Trade: Net trade: Value (goods): Total for Germany Monthly DEUXTNTVA01CXMLM
International Trade: Net trade: Value 

Employment by Economic Activity: Manufacturing: All Persons for the Republic of Korea Annual LFEAMNTTKRA647N
Employment by Economic Activity: Manufacturing: All Persons for the Republic of Korea Quarterly LFEAMNTTKRQ647S
Employment by Economic Activity: Manufacturing: All Persons for the Republic of Korea Quarterly LFEAMNTTKRQ647N
Early Estimate of Quarterly ULC Indicators: Total Labor Productivity for the Republic of Korea Quarterly ULQELP01KRQ661S
Early Estimate of Quarterly ULC Indicators: Total Labor Productivity for the Republic of Korea Quarterly ULQELP01KRQ659S
Early Estimate of Quarterly ULC Indicators: Total Labor Productivity for the Republic of Korea Quarterly ULQELP01KRQ661N
Stock Volume of Investment Goods for the Republic of Korea (DISCONTINUED) Quarterly STMNVG01KRQ661N
Stock Volume of Investment Goods for the Republic of Korea (DISCONTINUED) Annual STMNVG01KRA661N
Stock Volume of Investment Goods for the Republic of Korea (DISCONTINUED) Annual STMNVG01KRA661S
Financial 

Production: Construction: Total construction: Total for the Republic of Korea Quarterly KORPRCNTO01IXOBQ
Purchasing Power Parity Converted GDP Per Capita, G-K method, at current prices for Republic of Korea Annual PPCGDPKRA620NUPN
Total Employment by Professional Status: Employees for the Republic of Korea Monthly LFESEETTKRM647S
Total Employment by Professional Status: Employees for the Republic of Korea Monthly LFESEETTKRM647N
Total Employment by Professional Status: Employees for the Republic of Korea Annual LFESEETTKRA647S
Total Employment by Professional Status: Employees for the Republic of Korea Annual LFESEETTKRA647N
Total Employment by Professional Status: Employees for the Republic of Korea Quarterly LFESEETTKRQ647N
Total Employment by Professional Status: Employees for the Republic of Korea Quarterly LFESEETTKRQ647S
Employed Population: Aged 25-54: Females for the Republic of Korea Monthly LFEM25FEKRM647N
Employed Population: Aged 25-54: Females for the Republic of Korea Mon

...

Working Age Population: Aged 15-64: All Persons for the United States
USA
usa
Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level Weekly, As of Wednesday WALCL
U.S. / Euro Foreign Exchange Rate Daily DEXUSEU
U.S. / Euro Foreign Exchange Rate Monthly EXUSEU
U.S. / Euro Foreign Exchange Rate Annual AEXUSEU
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANS
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCI
Assets: Securities Held Outright: U.S. Treasury Securities: All: Wednesday Level Weekly, As of Wednesday TREAST
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCINSA
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANSNSA
M2 Money Stock Weekly, Ending Monday M2
10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity Daily T10Y2Y
Unemployment Rate Monthly UNRATE
Consumer Price Index for All Urban Consumers: All Items in U.S. 

Hong Kong / U.S. Foreign Exchange Rate Daily DEXHKUS
S&P/Case-Shiller NY-New York Home Price Index Monthly NYXRNSA
Total Assets, All Commercial Banks Quarterly TLAACBQ158SBOG
Gross National Product Annual GNPA
Norway / U.S. Foreign Exchange Rate Annual AEXNOUS
Consumer Price Index for All Urban Consumers: Rent of Primary Residence in U.S. City Average Monthly CUSR0000SEHA
Total Construction Spending Monthly TTLCON
Labor Force Participation Rate - Women Monthly LNS11300002
Institutional Money Funds Monthly IMFSL
Total Assets, All Commercial Banks Annual H8B1151NCBCAG
Advance Retail Sales: Retail (Excluding Food Services) Monthly RSXFSN
Manufacturing Sector: Real Output Quarterly PRS30006042
Total Consumer Credit Owned and Securitized, Outstanding Monthly, End of Period TOTALNS
Industrial Production: Manufacturing (NAICS) Monthly IPGMFN
Average Hourly Earnings of Production and Nonsupervisory Employees, Total Private Monthly CEU0500000008
Advance Retail Sales: Retail and Food Services, T

Real Imports of Goods and Services Quarterly ND000342Q
Personal income per capita Annual A792RC0A052NBEA
Trade Weighted U.S. Dollar Index: Emerging Markets Economies, Goods and Services Monthly TWEXEMEGSMTH
Personal Consumption Expenditures: Nondurable Goods Quarterly NA000348Q
6-Month Treasury Constant Maturity Rate Monthly GS6M
Total Business Inventories Monthly, End of Period TOTBUSIMNSA
Real Imports of Goods and Services Quarterly A021RO1Q156NBEA
Real Imports of Goods and Services Quarterly A021RL1Q158SBEA
3-Month AA Financial Commercial Paper Rate Weekly, Ending Friday WCPF3M
Personal Consumption Expenditures: Services Quarterly NA000350Q
Personal income per capita Quarterly A792RC0Q052SBEA
Unemployment Rate - White Monthly LNU04000003
Consumer Price Index for All Urban Consumers: Housing in U.S. City Average Monthly CPIHOSSL
Total Business Inventories Monthly TOTBUSMPCIMNSA
Personal Consumption Expenditures: Nondurable Goods Quarterly NC000348Q
Personal Consumption Expenditures: 

Share of Government Consumption at Current Purchasing Power Parities for China Annual CSHGCPCNA156NRUG
External Trade: Volume of Imports of Goods and Services for China, P.R.: Hong Kong Annual HKGTMRPCPPPT
Use of Financial Services, Liabilities: Outstanding Deposits at Commercial Banks for China, P.R.: Macao Annual MACFCLODCXDC
Use of Financial Services, Assets: Outstanding Loans to Households at Non-deposit Taking Microfinance Institutions (MFIs) for China, P.R.: Mainland Annual CHNFCSMMFHXDC
Geographical Outreach: Key Indicators Commercial Bank Branches Per 100,000 Adults for China, P.R.: Hong Kong Annual HKGFCBODCANUM
Use of Financial Services: Key Indicators, Deposit Accounts with Commercial Banks Per 1000 Adults for China, P.R.: Mainland Annual CHNFCAODCANUM
Geographical Outreach: Number of Branches, Excluding Headquarters, for Commercial Banks for China, P.R.: Macao Annual MACFCBODCNUM
Geographical Outreach: Number of Branches in 3 Largest Cities, Excluding Headquarters, for Othe

Value of Exports to China from Virginia Annual VACHNA052SCEN
Value of Exports to China from Mississippi Annual MSCHNA052SCEN
Number of Identified Exporters to China from Wisconsin Annual WICHNA475SCEN
Number of Identified Exporters to China from Montana Annual MTCHNA475SCEN
Number of Identified Exporters to China from U.S. Virgin Islands Annual VICHNA475SCEN
Number of Identified Exporters to China from Oklahoma Annual OKCHNA475SCEN
Net Issues of International Debt Securities for Issuers in General Government Sector, All Maturities, Nationality of Issuer in China (DISCONTINUED) Quarterly IDSGGGAMNINICN
Geographical Outreach: Key Indicators ATMs Per 1000 Km2 for China, P.R.: Hong Kong Annual HKGFCAKNUM
Geographical Outreach: Number of Automated Teller Machines (ATMs), Country Wide for China, P.R.: Hong Kong Annual HKGFCACNUM
Number of Identified Exporters to China from Nevada Annual NVCHNA475SCEN
Number of Identified Exporters to China from Maryland Annual MDCHNA475SCEN
Use of Financial 

In [13]:
working_age_population_ratio = {}
for country in working_age_population_ratio_set:
    working_age_population_ratio[country] = get_series_info_by_title(working_age_population_ratio_set[country], country )

Employment to Population Ratio for Germany
Germany
germany
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Monthly IRLTLT01DEM156N
Real Gross Domestic Product for Germany Quarterly CLVMNACSCAB1GQDE
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Quarterly IRLTLT01DEQ156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Germany Annual IRLTLT01DEA156N
Real Gross Domestic Product for Germany Quarterly CLVMNACNSAB1GQDE
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156S
Registered Unemployment Rate for Germany Quarterly LMUNRRTTDEQ156S
Registered Unemployment Rate for Germany Monthly LMUNRRTTDEM156N
Consumer Price Index of All Items in Germany Monthly DEUCPIALLMINMEI
Registered Unemployment Rate for Germany Annual LMUNRRTTDEA156N
Registered Unemployment Rate for Germany Quarterly LMUNRRTTDEQ156N
Registered Unemployment Rate for Germany Annual LMUNRRTTDEA156S
Consumer Price Index of All

Total Credit to Non-Financial Sector, Adjusted for Breaks, for Germany Quarterly, End of Quarter QDECAM770A
Active Population: Aged 15 and Over: All Persons for Germany Annual LFACTTTTDEA647N
Unemployment Rate: Aged 15-24: All Persons for Germany Quarterly LRUN24TTDEQ156N
Consumer Price Index: OECD Groups: Housing: Total for Germany Monthly DEUCPGRHO01GYM
Consumer Price Index: OECD Groups: Housing: Total for Germany Monthly DEUCPGRHO01GPM
Consumer Price Index: OECD Groups: Housing: Total for Germany Quarterly DEUCPGRHO01GPQ
Consumer Price Index: OECD Groups: Housing: Total for Germany Monthly DEUCPGRHO01IXOBM
Consumer Price Index: OECD Groups: Housing: Total for Germany Quarterly DEUCPGRHO01GYQ
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Quarterly DEUULCCONQPNMEI
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Quarterly ULQBBU04DEQ661S
Benchmarked Unit Labor Costs - Construction for Germany (DISCONTINUED) Annual ULQBBU04DEA662N
Benchmar

Capital stock at Current Purchasing Power Parities for Germany Annual CKSPPPDEA666NRUG
Consumer Price Index: Communication (COICOP 08): Total: Total for Germany Quarterly DEUCP080000GYQ
Consumer Price Index: Communication (COICOP 08): Total: Total for Germany Monthly DEUCP080000IXOBM
Consumer Price Index: Communication (COICOP 08): Total: Total for Germany Monthly DEUCP080000GPM
Consumer Price Index: Communication (COICOP 08): Total: Total for Germany Monthly DEUCP080000GYM
Consumer Price Index: Communication (COICOP 08): Total: Total for Germany Monthly DEUCP080000IXNBM
Consumer Price Index: Communication (COICOP 08): Total: Total for Germany Quarterly DEUCP080000IXNBQ
Inactivity Rate: Aged 15 and Over: All Persons for Germany Quarterly LRINTTTTDEQ156S
Inactivity Rate: Aged 15 and Over: All Persons for Germany Annual LRINTTTTDEA156N
Harmonized Index of Consumer Prices: Restaurants, cafés, and the Like for Germany (including former GDR from 1991) Monthly CP1111DEM086NEST
Private Discou

Active Population: Aged 15 and Over: All Persons for the Republic of Korea Annual LFACTTTTKRA647S
Active Population: Aged 15 and Over: All Persons for the Republic of Korea Annual LFACTTTTKRA647N
Active Population: Aged 15 and Over: All Persons for the Republic of Korea Annual LFACTTTTKRA657N
Active Population: Aged 15 and Over: All Persons for the Republic of Korea Annual LFACTTTTKRA657S
Active Population: Aged 15 and Over: All Persons for the Republic of Korea Quarterly LFACTTTTKRQ647N
Active Population: Aged 15 and Over: All Persons for the Republic of Korea Quarterly LFACTTTTKRQ657S
Active Population: Aged 15 and Over: All Persons for the Republic of Korea Quarterly LFACTTTTKRQ657N
Goods, Value of Imports for Republic of Korea Monthly VALIMPKRM052N
Goods, Value of Imports for Republic of Korea Quarterly VALIMPKRQ052N
Employment Rate: Aged 15-64: Males for the Republic of Korea Monthly LREM64MAKRM156S
Employment Rate: Aged 15-64: Males for the Republic of Korea Monthly LREM64MAKRM15

Employment to Population Ratio for Japan
Japan
japan
Bank of Japan: Total Assets for Japan Monthly, End of Period JPNASSETS
Japan / U.S. Foreign Exchange Rate Daily DEXJPUS
Japan / U.S. Foreign Exchange Rate Monthly EXJPUS
Japan / U.S. Foreign Exchange Rate Annual AEXJPUS
Real Gross Domestic Product for Japan Quarterly JPNRGDPEXP
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Japan Monthly IRLTLT01JPM156N
Gross Domestic Product for Japan Quarterly JPNNGDP
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Japan Quarterly IRLTLT01JPQ156N
Long-Term Government Bond Yields: 10-year: Main (Including Benchmark) for Japan Annual IRLTLT01JPA156N
Gross Domestic Product for Japan Annual MKTGDPJPA646NWDB
Residential Property Prices for Japan Quarterly QJPN628BIS
Residential Property Prices for Japan Quarterly QJPN368BIS
Consumer Price Index of All Items in Japan Monthly JPNCPIALLMINMEI
Interest Rates, Discount Rate for Japan Monthly INTDSRJPM193N


Gross Domestic Product by Expenditure in Constant Prices: Gross Fixed Capital Formation for Japan Annual NAEXKP04JPA659S
Gross Domestic Product by Expenditure in Constant Prices: Gross Fixed Capital Formation for Japan Annual NAEXKP04JPA657S
Sales: Retail trade: Car registration: Passenger cars for Japan Monthly JPNSLRTCR03MLM
Sales: Retail trade: Car registration: Passenger cars for Japan Monthly JPNSLRTCR03GYSAM
Sales: Retail trade: Car registration: Passenger cars for Japan Monthly JPNSLRTCR03GPSAM
Sales: Retail trade: Car registration: Passenger cars for Japan Quarterly JPNSLRTCR03GYSAQ
Monthly Overtime Hours: Manufacturing for Japan Quarterly HOOVMN03JPQ661N
Employment Rate: Aged 25-54: All Persons for Japan Monthly LREM25TTJPM156N
Employment Rate: Aged 25-54: All Persons for Japan Quarterly LREM25TTJPQ156N
Employment Rate: Aged 25-54: All Persons for Japan Quarterly LREM25TTJPQ156S
Employment Rate: Aged 25-54: All Persons for Japan Annual LREM25TTJPA156S
Employment Rate: Aged 25-

Employment to Population Ratio for the United States
USA
usa
Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level Weekly, As of Wednesday WALCL
U.S. / Euro Foreign Exchange Rate Daily DEXUSEU
U.S. / Euro Foreign Exchange Rate Monthly EXUSEU
U.S. / Euro Foreign Exchange Rate Annual AEXUSEU
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANS
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCI
Assets: Securities Held Outright: U.S. Treasury Securities: All: Wednesday Level Weekly, As of Wednesday TREAST
Commercial and Industrial Loans, All Commercial Banks Weekly, Ending Wednesday TOTCINSA
Commercial and Industrial Loans, All Commercial Banks Monthly BUSLOANSNSA
M2 Money Stock Weekly, Ending Monday M2
10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity Daily T10Y2Y
Unemployment Rate Monthly UNRATE
Consumer Price Index for All Urban Consumers: All Items in U.S. City Average Mont

Employment Level Monthly CE16OV
Net Interest Margin for all U.S. Banks Quarterly, End of Period USNIM
Federal Receipts as Percent of Gross Domestic Product Annual FYFRGDA188S
Consumer Loans: Credit Cards and Other Revolving Plans, All Commercial Banks Monthly H8B1247NCBCMG
Nonfinancial Corporate Business; Debt Securities and Loans; Liability, Level Annual BOGZ1LA104104005A
Bank Credit, All Commercial Banks Monthly H8B1001NCBCMG
Net Exports of Goods and Services Quarterly NETEXP
Government total expenditures Annual W068RC1A027NBEA
Federal government current tax receipts Quarterly W006RC1Q027SBEA
Federal Funds Target Range - Lower Limit Daily, 7-Day DFEDTARL
ICE BofA Single-A US Corporate Index Effective Yield Daily, Close BAMLC0A3CAEY
Treasury Inflation-Indexed Long-Term Average Yield Daily DLTIIT
Personal Consumption Expenditures: Durable Goods Monthly PCEDG
Dates of U.S. recessions as inferred by GDP-based recession indicator Quarterly JHDUSRGDPBR
Gross Private Domestic Investment Ann

China / U.S. Foreign Exchange Rate Daily DEXCHUS
China / U.S. Foreign Exchange Rate Monthly EXCHUS
China / U.S. Foreign Exchange Rate Annual AEXCHUS
Gross Domestic Product for China Annual MKTGDPCNA646NWDB
Consumer Price Index: All Items for China Monthly CHNCPIALLMINMEI
M2 for China Monthly MYAGM2CNM189N
Consumer Price Index: All Items for China Annual CHNCPIALLAINMEI
Consumer Price Index: All Items for China Quarterly CHNCPIALLQINMEI
Current Price Gross Domestic Product in China Quarterly CHNGDPNQDSMEI
Current Price Gross Domestic Product in China Annual CHNGDPNADSMEI
Real GDP at Constant National Prices for China Annual RGDPNACNA666NRUG
M1 for China Monthly MYAGM1CNM189N
M1 for China Monthly MANMM101CNM189S
M1 for China Monthly MANMM101CNM189N
M1 for China Quarterly MANMM101CNQ189N
M1 for China Monthly MANMM101CNM657S
M1 for China Annual MANMM101CNA189S
M1 for China Annual MANMM101CNA657S
M1 for China Quarterly MANMM101CNQ189S
M1 for China Quarterly MANMM101CNQ657S
M1 for China Annu

Price Level of Government Consumption for China Annual PLGCPPCNA670NRUG
Geographical Outreach: Number of Branches, Excluding Headquarters, for Commercial Banks for China, P.R.: Mainland Annual CHNFCBODCNUM
Share of Government Consumption at Current Purchasing Power Parities for China Annual CSHGCPCNA156NRUG
External Trade: Volume of Imports of Goods and Services for China, P.R.: Hong Kong Annual HKGTMRPCPPPT
Use of Financial Services, Liabilities: Outstanding Deposits at Commercial Banks for China, P.R.: Macao Annual MACFCLODCXDC
Use of Financial Services, Assets: Outstanding Loans to Households at Non-deposit Taking Microfinance Institutions (MFIs) for China, P.R.: Mainland Annual CHNFCSMMFHXDC
Geographical Outreach: Key Indicators Commercial Bank Branches Per 100,000 Adults for China, P.R.: Hong Kong Annual HKGFCBODCANUM
Use of Financial Services: Key Indicators, Deposit Accounts with Commercial Banks Per 1000 Adults for China, P.R.: Mainland Annual CHNFCAODCANUM
Geographical Outreac

In [14]:
working_age_population

{'Germany': {'Germany': {'title': 'Working Age Population: Aged 15-64: All Persons for Germany',
   'units': 'lin',
   'notes': 'OECD descriptor ID: LFWA64TT\nOECD unit ID: ST\nOECD country ID: DEU\n\nAll OECD data should be cited as follows: OECD, "Main Economic Indicators - complete database", Main Economic Indicators (database),http://dx.doi.org/10.1787/data-00052-en (Accessed on date)\nCopyright, 2016, OECD. Reprinted with permission.',
   'dates': ['2005-01-01',
    '2006-01-01',
    '2007-01-01',
    '2008-01-01',
    '2009-01-01',
    '2010-01-01',
    '2011-01-01',
    '2012-01-01',
    '2013-01-01',
    '2014-01-01',
    '2015-01-01',
    '2016-01-01',
    '2017-01-01',
    '2018-01-01',
    '2019-01-01'],
   'values': ['54764150.0',
    '54543175.0',
    '54228900.0',
    '54065975.0',
    '53762500.0',
    '52399600.0',
    '52314000',
    '52486950.0',
    '52576625.0',
    '52729325.0',
    '52963825.0',
    '53802300.0',
    '53797375.0',
    '53523600.0',
    '53545050.0

In [15]:
calculated_working_age_population = []
calculated_working_age_dates = []
for date in working_age_population['China']['China']['dates']:
    if date in total_population_set['China']['China']['dates']:
        calculated_working_age_dates.append(date)
        working_population_index = working_age_population['China']['China']['dates'].index(date)
        total_population_index = total_population_set['China']['China']['dates'].index(date)
        print(date, working_population_index, total_population_index)
        total_population_value = total_population_set['China']['China']['values'][total_population_index]
        working_population_value = working_age_population['China']['China']['values'][working_population_index]
        print(total_population_value, working_population_value)
        if  working_population_value.replace('.','',1).isnumeric() and working_population_value.replace('.','',1).isnumeric():
            print(total_population_value,working_population_value )
            calculated_working_age_population.append(
                str( float(total_population_value) * (float(working_population_value) /100)  ))
        else:
            calculated_working_age_population.append('')
        
        
     

1952-01-01 2 0
577.6072387695312 .
1953-01-01 3 1
587.31396484375 .
1954-01-01 4 2
597.9037475585938 .
1955-01-01 5 3
609.9727783203125 .
1956-01-01 6 4
622.4453125 .
1957-01-01 7 5
636.5991821289062 .
1958-01-01 8 6
650.1597900390625 .
1959-01-01 9 7
657.8470458984375 .
1960-01-01 10 8
654.138427734375 56.4892986405363
654.138427734375 56.4892986405363
1961-01-01 11 9
648.1157836914062 56.4143062276544
648.1157836914062 56.4143062276544
1962-01-01 12 10
656.7941284179688 56.0432071621243
656.7941284179688 56.0432071621243
1963-01-01 13 11
677.8527221679688 55.5933222615383
677.8527221679688 55.5933222615383
1964-01-01 14 12
699.7855224609375 55.3484291751092
699.7855224609375 55.3484291751092
1965-01-01 15 13
719.3711547851562 55.3978086362411
719.3711547851562 55.3978086362411
1966-01-01 16 14
739.8373413085938 55.1412106991901
739.8373413085938 55.1412106991901
1967-01-01 17 15
759.357421875 55.220927981574
759.357421875 55.220927981574
1968-01-01 18 16
780.3014526367188 55.48364218

In [16]:
calculated_working_age_population

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '369.5182099653798',
 '365.6300229214321',
 '368.0884940179509',
 '376.84084829344874',
 '387.3202942769599',
 '398.5158557121986',
 '407.9552672022579',
 '419.3242150563308',
 '432.93966594069644',
 '447.37291309620565',
 '460.6010616127814',
 '471.4316680056916',
 '482.3156895242123',
 '493.15420251079735',
 '504.37933773702844',
 '516.4129560202813',
 '529.5874546205473',
 '543.3112868245718',
 '557.8971711607721',
 '573.5501422714932',
 '590.2652001605522',
 '608.6882121539433',
 '627.0928983185718',
 '645.8644123602506',
 '665.1952274569779',
 '684.8581043076135',
 '702.9709142522325',
 '721.1079695347405',
 '738.6775552581629',
 '755.3952183252115',
 '771.2358641398494',
 '781.9292420150477',
 '794.7172382034441',
 '808.0100154132676',
 '819.6410873013277',
 '828.8197092223701',
 '842.3483989125855',
 '850.713824647639',
 '856.9141484569983',
 '865.2885900312424',
 '877.7007385407497',
 '888.7551242858525',
 '904.3541623786386',
 '922.5186

In [17]:
working_age_population['China']['China']['values'] = calculated_working_age_population
working_age_population['China']['China']['dates'] = calculated_working_age_dates

In [18]:
working_age_population['China']['China']['dates']

['1952-01-01',
 '1953-01-01',
 '1954-01-01',
 '1955-01-01',
 '1956-01-01',
 '1957-01-01',
 '1958-01-01',
 '1959-01-01',
 '1960-01-01',
 '1961-01-01',
 '1962-01-01',
 '1963-01-01',
 '1964-01-01',
 '1965-01-01',
 '1966-01-01',
 '1967-01-01',
 '1968-01-01',
 '1969-01-01',
 '1970-01-01',
 '1971-01-01',
 '1972-01-01',
 '1973-01-01',
 '1974-01-01',
 '1975-01-01',
 '1976-01-01',
 '1977-01-01',
 '1978-01-01',
 '1979-01-01',
 '1980-01-01',
 '1981-01-01',
 '1982-01-01',
 '1983-01-01',
 '1984-01-01',
 '1985-01-01',
 '1986-01-01',
 '1987-01-01',
 '1988-01-01',
 '1989-01-01',
 '1990-01-01',
 '1991-01-01',
 '1992-01-01',
 '1993-01-01',
 '1994-01-01',
 '1995-01-01',
 '1996-01-01',
 '1997-01-01',
 '1998-01-01',
 '1999-01-01',
 '2000-01-01',
 '2001-01-01',
 '2002-01-01',
 '2003-01-01',
 '2004-01-01',
 '2005-01-01',
 '2006-01-01',
 '2007-01-01',
 '2008-01-01',
 '2009-01-01',
 '2010-01-01',
 '2011-01-01',
 '2012-01-01',
 '2013-01-01',
 '2014-01-01',
 '2015-01-01',
 '2016-01-01',
 '2017-01-01']

In [19]:
#working_age_population_ratio

In [20]:
from collections import defaultdict

In [21]:
def make_dataframe(dict_info):
    columns = dict_info.keys()
    values = defaultdict(list)
    dates_set = set()
    for country in columns:
        #print(country)
        #print(dict_info[country])
        #print(dict_info[country][country])
        dates_set.update(set(dict_info[country][country]['dates']))
   
    print (dates_set)
    
    dates_list = list(dates_set)
    dates_list = sorted(dates_list)
    for date in dates_list:
        for country in dict_info:
            print(date, country)
            if date in dict_info[country][country]['dates']:
                values[date].append(
                    dict_info[country][country]['values'][dict_info[country][country]['dates'].index(date)])
            else:
                values[date].append('')
    print(values)
    df_ans = pd.DataFrame.from_dict(values, orient='index', columns=columns)         
    return df_ans
    

In [22]:
df_countries = make_dataframe(working_age_population)

{'1953-01-01', '2012-01-01', '1983-01-01', '2003-01-01', '1987-01-01', '1962-01-01', '1966-01-01', '2015-01-01', '2017-01-01', '1965-01-01', '1956-01-01', '1992-01-01', '1991-01-01', '1952-01-01', '2000-01-01', '1976-01-01', '1977-01-01', '2002-01-01', '1985-01-01', '1971-01-01', '2013-01-01', '1986-01-01', '1984-01-01', '1963-01-01', '2004-01-01', '1975-01-01', '1968-01-01', '2018-01-01', '2005-01-01', '1982-01-01', '1960-01-01', '1993-01-01', '1970-01-01', '1954-01-01', '1964-01-01', '2007-01-01', '1996-01-01', '1961-01-01', '2006-01-01', '2011-01-01', '1990-01-01', '1979-01-01', '2008-01-01', '1997-01-01', '1959-01-01', '2009-01-01', '2010-01-01', '1989-01-01', '1958-01-01', '1957-01-01', '1995-01-01', '1988-01-01', '1981-01-01', '1969-01-01', '2014-01-01', '1999-01-01', '2019-01-01', '2001-01-01', '1974-01-01', '1978-01-01', '1973-01-01', '1967-01-01', '1998-01-01', '1980-01-01', '1955-01-01', '1994-01-01', '1972-01-01', '2016-01-01'}
1952-01-01 Germany
1952-01-01 South Korea
1952-

In [23]:

df_countries.rename(columns={'USA':'United States'}, inplace=True)

In [24]:
# Working age population
df_countries

,Germany,South Korea,Japan,United States,China
1952-01-01,,,,,
1953-01-01,,,,,
1954-01-01,,,,,
1955-01-01,,,,.,
1956-01-01,,,,.,
...,...,...,...,...,...
2015-01-01,52963825.0,36705075.0,77256666.6666666,204291750.0,1014.4429846960028
2016-01-01,53802300.0,36838591.6666667,76496666.6666666,205502750.0,1013.3658995926229
2017-01-01,53797375.0,36859450.0,75985833.3333333,205537500.0,1010.8709414641698
2018-01-01,53523600.0,36796125.0,75454166.6666667,206507750.0,


In [25]:
df_countries.head()

,Germany,South Korea,Japan,United States,China
1952-01-01,,,,,
1953-01-01,,,,,
1954-01-01,,,,,
1955-01-01,,,,.,
1956-01-01,,,,.,


In [26]:
for column in list(df_countries.columns):
    df_countries[column] = pd.to_numeric(df_countries[column],  errors='coerce')
    print(column)
    if column != 'China':
        df_countries[column] = df_countries[column] / 1000000


Germany
South Korea
Japan
United States
China


In [27]:
df_countries_total = make_dataframe(total_population_set)

{'1934-01-01', '1939-01-01', '1953-01-01', '2003-01-01', '1966-01-01', '1929-01-01', '1965-01-01', '2015-01-01', '1956-01-01', '1937-01-01', '1935-01-01', '1938-01-01', '1952-01-01', '1971-01-01', '1930-01-01', '2013-01-01', '1984-01-01', '2004-01-01', '1975-01-01', '2018-01-01', '1948-01-01', '2005-01-01', '1982-01-01', '1944-01-01', '1970-01-01', '1964-01-01', '2011-01-01', '1979-01-01', '2009-01-01', '1950-01-01', '1945-01-01', '1989-01-01', '1958-01-01', '1988-01-01', '2014-01-01', '2019-01-01', '1978-01-01', '1980-01-01', '1994-01-01', '1972-01-01', '1951-01-01', '2016-01-01', '1936-01-01', '1983-01-01', '2012-01-01', '1943-01-01', '1962-01-01', '1987-01-01', '2017-01-01', '1992-01-01', '1991-01-01', '1940-01-01', '1946-01-01', '2000-01-01', '1976-01-01', '1932-01-01', '1977-01-01', '2002-01-01', '1985-01-01', '1986-01-01', '1963-01-01', '1933-01-01', '1968-01-01', '1931-01-01', '1954-01-01', '1960-01-01', '1993-01-01', '1996-01-01', '2007-01-01', '1961-01-01', '2006-01-01', '1990

2012-01-01 Republic of Korea
2012-01-01 Germany
2012-01-01 United States
2013-01-01 China
2013-01-01 Japan
2013-01-01 Republic of Korea
2013-01-01 Germany
2013-01-01 United States
2014-01-01 China
2014-01-01 Japan
2014-01-01 Republic of Korea
2014-01-01 Germany
2014-01-01 United States
2015-01-01 China
2015-01-01 Japan
2015-01-01 Republic of Korea
2015-01-01 Germany
2015-01-01 United States
2016-01-01 China
2016-01-01 Japan
2016-01-01 Republic of Korea
2016-01-01 Germany
2016-01-01 United States
2017-01-01 China
2017-01-01 Japan
2017-01-01 Republic of Korea
2017-01-01 Germany
2017-01-01 United States
2018-01-01 China
2018-01-01 Japan
2018-01-01 Republic of Korea
2018-01-01 Germany
2018-01-01 United States
2019-01-01 China
2019-01-01 Japan
2019-01-01 Republic of Korea
2019-01-01 Germany
2019-01-01 United States
defaultdict(<class 'list'>, {'1929-01-01': ['', '', '', '', '121878'], '1930-01-01': ['', '', '', '', '123188'], '1931-01-01': ['', '', '', '', '124149'], '1932-01-01': ['', '', 

In [28]:
df_countries_total.rename(columns={'Republic of Korea': 'South Korea'}, inplace=True)

In [29]:
df_countries_total

,China,Japan,South Korea,Germany,United States
1929-01-01,,,,,121878
1930-01-01,,,,,123188
1931-01-01,,,,,124149
1932-01-01,,,,,124949
1933-01-01,,,,,125690
...,...,...,...,...,...
2015-01-01,1397.028564453125,,,,320918.0
2016-01-01,1403.5003662109375,,,,323186.0
2017-01-01,1409.5174560546875,,,,325220.0
2018-01-01,,,,,326949.0


In [30]:
for column in list(df_countries_total.columns):
    df_countries_total[column] = pd.to_numeric(df_countries_total[column],  errors='coerce')
    print(column)
    if column == 'China':
        df_countries_total[column] = df_countries_total[column]
    else:
        df_countries_total[column] = df_countries_total[column] / 1000

China
Japan
South Korea
Germany
United States


In [31]:
# Change year labels to only year
df_countries['Year'] = df_countries.index.map(lambda x: x.split('-')[0])
df_countries_total['Year'] = df_countries_total.index.map(lambda x: x.split('-')[0])

In [32]:
df_countries.set_index(['Year'], inplace=True)
df_countries_total.set_index(['Year'], inplace=True)

In [33]:
data_dir = '../data'

In [34]:
import os
pop_file = 'FRED_pop.csv'
work_file = 'FRED_work.csv'
pop_path = os.path.join(data_dir, pop_file)
work_path = os.path.join(data_dir, work_file)

In [35]:
df_countries.to_csv(work_path, index=True)
df_countries_total.to_csv(pop_path, index=True)

In [36]:
#

In [37]:
df_countries

,Germany,South Korea,Japan,United States,China
Year,,,,,
1952,NaN,NaN,NaN,NaN,NaN
1953,NaN,NaN,NaN,NaN,NaN
1954,NaN,NaN,NaN,NaN,NaN
1955,NaN,NaN,NaN,NaN,NaN
1956,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2015,52.963825,36.705075,77.256667,204.291750,1014.442985
2016,53.802300,36.838592,76.496667,205.502750,1013.365900
2017,53.797375,36.859450,75.985833,205.537500,1010.870941


In [ ]:
df_countries.plot(title='Working Age Population')

In [ ]:
df_temp = df_countries.copy()

In [ ]:
# Normalize data
df_temp2 = (df_temp - df_temp.min()) / (df_temp.max() - df_temp.min())

In [ ]:
df_temp2.plot(title='Working Age Population  - normalized')

In [ ]:
df_temp3 = (df_temp / df_temp.max()) * 100

In [ ]:
df_temp3.plot(title='Working Age Population  - max = 100%')

In [ ]:
df_countries_total.plot(title='Total Population')

In [ ]:
df_countries_total